# ⭐ Tutorial: Detecting Backtest Overfitting

This notebook is a capstone tutorial for the `RiskLabAI` backtesting module. It demonstrates the complete workflow for detecting backtest overfitting, as described by De Prado (Chapters 10-14, 'Advances in Financial Machine Learning').

The central problem in strategy selection is: **Is my 'best' strategy genuinely good, or did I just get lucky?**

We will answer this by:
1.  **Generating Synthetic Data:** Create a market environment and a set of (intentionally bad) trading strategies using `backtest_synthetic_data`.
2.  **Running Simulations:** Execute all strategies and find the 'best' one using `backtest_overfitting_simulation`.
3.  **Calculating PBO:** Use `get_pbo` to find the Probability of Backtest Overfitting. This tells us the likelihood that our 'best' strategy was a false discovery (a fluke).
4.  **Calculating PSR:** Use `probabilistic_sharpe_ratio` to check if the 'best' strategy's Sharpe Ratio is statistically significant, given its skew and kurtosis.
5.  **Running CSCV:** Use `combinatorial_symmetric_cross_validation` to find the combination of parameters that performs best *on average* across all tests, helping us find the *most robust* strategy, not just the one with the highest single SR.

## 0. Setup and Imports

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# RiskLabAI Imports
from RiskLabAI.backtest.backtest_overfitting_simulation import run_backtest_overfitting_simulation
from RiskLabAI.backtest.probabilistic_sharpe_ratio import probabilistic_sharpe_ratio, compute_psr_curve
from RiskLabAI.backtest.probability_of_backtest_overfitting import get_pbo, pbo_overfitting_plot
from RiskLabAI.backtest.test_set_overfitting import combinatorial_symmetric_cross_validation
from RiskLabAI.backtest.backtest_statistics import annualized_sharpe_ratio
import RiskLabAI.utils.publication_plots as pub_plots

# Setup plotting and configuration
pub_plots.setup_publication_style()
warnings.filterwarnings('ignore')

## 1. Run the Backtest Overfitting Simulation

First, we call the main orchestration function: `run_backtest_overfitting_simulation`.

This function does a lot of work (using other modules we've built):
1.  It calls `generate_market_data` to create a synthetic price history.
2.  It generates `n_strategies` (100) different strategies with *no actual edge* (signal `t_value = 0.0`).
3.  It 'backtests' all 100 of these strategies on the *same* price history.
4.  It returns a DataFrame of the Sharpe Ratios (SR) for all 100 strategies, plus the SR for the single 'best' one.

We expect the 'best' strategy to have a high SR *purely by luck*.

In [ ]:
n_strategies = 100
strategy_sr_mean = 0.0 # Strategies have no edge
strategy_sr_std = 0.5   # High variance in performance
n_trials = 1000       # Number of simulations to run
n_years = 10          # 10 years of data
obs_per_year = 260    # Daily observations

# Run the main simulation engine
results = run_backtest_overfitting_simulation(
    n_trials=n_trials,
    n_strategies=n_strategies,
    n_years=n_years,
    obs_per_year=obs_per_year,
    strategy_sr_mean=strategy_sr_mean,
    strategy_sr_std=strategy_sr_std,
    signal_t_value=0.0 # No true signal
)

print(f"Ran {n_trials} trials of {n_strategies} strategies each.")
print(f"Average SR of the 'best' strategy (SR_star): {results['SR_star'].mean():.4f}")
print(f"Average SR of all strategies (SR_all): {results['SR_all'].mean():.4f}")

## 2. Probability of Backtest Overfitting (PBO)

The simulation shows that the *best* strategy (SR_star) had an average SR of ~1.98, while the average strategy (SR_all) had an SR of 0.0. This high SR_star is a classic example of overfitting.

Now, we use `get_pbo` to calculate the probability that our `SR_star` was a fluke.

We also use the `pbo_overfitting_plot` helper to visualize this. The plot shows the distribution of all SRs (blue) and the distribution of the *best* SRs (orange). The PBO value is the area of overlap.

In [ ]:
# Get the Sharpe Ratios for all strategies from all trials
sr_all_strategies = results['SR_all']
sr_best_strategies = results['SR_star']

# Calculate PBO
pbo = get_pbo(sr_all_strategies, sr_best_strategies)

print(f"Probability of Backtest Overfitting (PBO): {pbo * 100:.2f}%")

# Plot the PBO distributions
fig, ax = plt.subplots(figsize=(10, 6))
pbo_overfitting_plot(
    sr_all_strategies,
    sr_best_strategies,
    ax=ax
)

pub_plots.apply_plot_style(
    ax,
    title=f'Probability of Backtest Overfitting (PBO) = {pbo * 100:.2f}%',
    xlabel='Sharpe Ratio (SR)',
    ylabel='Density'
)
plt.show()

**Observation:** The PBO is extremely high (e.g., 98%). This correctly tells us that the 'best' strategies we found are almost certainly overfit and are just the positive tail of a random (no-edge) distribution.

## 3. Probabilistic Sharpe Ratio (PSR)

PBO works on a *set* of strategies. PSR works on a *single* strategy. Let's pretend we are a researcher who only has one strategy (our 'best' one) and wants to test if it's significant.

We'll use one of the return series from our simulation (from `results['returns_star']`) to calculate its PSR.

In [ ]:
# Get the returns of the 'best' strategy from the first trial
best_strategy_returns = results['returns_star'][0]

# Calculate its annualized SR (this is our 'claimed' SR)
claimed_sr = annualized_sharpe_ratio(best_strategy_returns, obs_per_year=obs_per_year)
n_years = best_strategy_returns.shape[0] / obs_per_year

print(f"'Best' Strategy Annualized SR: {claimed_sr:.4f}")

# Calculate PSR
# We test it against a benchmark of SR=0.0 (no edge)
psr = probabilistic_sharpe_ratio(
    observed_sr=claimed_sr,
    n_years=n_years,
    skew=best_strategy_returns.skew(),
    kurtosis=best_strategy_returns.kurtosis(),
    benchmark_sr=0.0
)

print(f"Probabilistic Sharpe Ratio (PSR): {psr:.4f}")

**Observation:** Even though our 'best' strategy had a high SR, its PSR is low (e.g., 0.28). This correctly signals that the strategy is *not* statistically significant and likely a fluke. A high PSR (e.g., > 0.95) would be needed to be confident.

We can also use `compute_psr_curve` to see what SR we *would* need to achieve 95% confidence.

In [ ]:
# Get the PSR curve for different 'claimed' SR values
psr_curve = compute_psr_curve(
    n_years=n_years,
    skew=best_strategy_returns.skew(),
    kurtosis=best_strategy_returns.kurtosis(),
    benchmark_sr=0.0
)

# Find the SR needed for PSR=0.95
required_sr = psr_curve[psr_curve > 0.95].idxmin()

fig, ax = plt.subplots(figsize=(10, 6))
psr_curve.plot(ax=ax, label='PSR Curve')
ax.axhline(0.95, ls='--', color='red', label='PSR = 0.95')
ax.axvline(required_sr, ls='--', color='green', label=f'Required SR = {required_sr:.2f}')

pub_plots.apply_plot_style(
    ax,
    title='Probabilistic Sharpe Ratio (PSR) Curve',
    xlabel='Claimed Sharpe Ratio',
    ylabel='PSR (Probability of Significance)'
)
ax.legend()
plt.show()

## 4. Combinatorial Symmetric Cross-Validation (CSCV)

Finally, we use CSCV. This method helps us find the *most robust* set of parameters, not just the one with the highest SR.

We'll test 6 combinations of parameters (e.g., 6 pairs of (Fast MA, Slow MA)). The `combinatorial_symmetric_cross_validation` function will:
1.  Split the data into $S=2$ splits (J=2).
2.  Test all 6 combinations on all $2^2 = 4$ possible train/test paths.
3.  Aggregate the performance and rank the combinations.

The combination with the *best average rank* (lowest score) is the most robust.

In [ ]:
# --- 1. Create a dummy performance matrix ---
# This simulates the performance of 6 parameter combinations (rows)
# across 2 different time splits (columns).
# Shape = (n_combinations, n_splits)
# We'll assume J=2, so n_splits = J * (J-1) / 2 = 1. Wait, S=2.
# Let's assume S=2 splits (J=2). We test all 2^S = 4 combinations.
# We will use S=2 splits (J=2), giving 2^S = 4 paths.
# Let's manually create a simple performance matrix for 6 parameter combinations (rows) 
# over S=4 splits (columns).
n_params = 6
n_splits = 4

np.random.seed(42)
performance_matrix = pd.DataFrame(np.random.rand(n_params, n_splits))
# Make one combination (idx 3) consistently good
performance_matrix.iloc[3, :] = [0.8, 0.75, 0.85, 0.7]
# Make one combination (idx 1) a 'fluke' (good once, bad otherwise)
performance_matrix.iloc[1, :] = [0.9, 0.2, 0.1, 0.3]

print("Performance Matrix (Combinations x Splits):")
display(performance_matrix)

# --- 2. Run CSCV --- 
cscv_results = combinatorial_symmetric_cross_validation(performance_matrix)

print("\nCSCV Results:")
display(cscv_results.sort_values(by='mean_rank'))

**Conclusion:** As expected, `combinatorial_symmetric_cross_validation` did *not* choose parameter combination 1, even though it had the highest performance (0.9) in one split. 

Instead, it chose **parameter combination 3**, which was *consistently* good across all splits and had the best average rank. This is the more robust and reliable model.